In [1]:
import anndata as ad
import pandas as pd
import numpy as np
import scanpy as sc
import decoupler as dc
import os

In [2]:
def validate_input_arguments (arguments_list):
    if arguments_list["out_path"] is None:
        print("Please provide an output path")
    elif arguments_list["out_path"][-1] != "/":
        arguments_list["out_path"] = arguments_list["out_path"] + "/"

    if arguments_list["celltype"] is None:
        print("Please provide the name of the metadata field containing cell type annotations")

    if arguments_list["condition"] is None:
        print("Please provide the name of the metadata field containing condition annotations")

    if arguments_list["organism"] is None:
        arguments_list["organism"] = "human"

    if arguments_list["comparison_list"] is None:
        arguments_list["comparison_list"] = np.nan

    if arguments_list["logfc"] is None:
        arguments_list["logfc"] = 0.0

    if arguments_list ["pval"] is None:
        arguments_list["pval"] = 0.05

    if arguments_list["reg"] is None:
        arguments_list["reg"] = load_dorothea_regulon(arguments_list["organism"])

    elif isinstance(arguments_list["reg"], str):
        arguments_list["reg"] = pd.read_csv(arguments_list["reg"], index_col=0)
        arguments_list["reg"] = pd.DataFrame.rename(arguments_list["reg"], columns={"source" : "tf"})

    if not "tf" in arguments_list["reg"] and "target" in arguments_list["reg"] and "weight"in arguments_list["reg"]:
        raise Exception("Not all necessary columns found in regulon table! Please make sure that the regulon has the columns source, target and weight!")
    
 
   
    return(arguments_list)



In [28]:
def AverageExpression(sub_object, celltype = None, name_iterable = None, outpath = None):
    gene_ids = sub_object.var.index.values
    #cluster line even necessary if not returned?
    #clusters = anndataobject.obs[celltype].cat.categories
    obs = sub_object[:,gene_ids].X.toarray()
    obs = np.expm1(obs)
    avg_df = pd.DataFrame(obs,columns=gene_ids,index= sub_object.obs[celltype])
    avg_df = avg_df.groupby(level=0, observed=False).mean()
    avg_df.T.to_csv(outpath + name_iterable + "_average_gene_expression_by_cluster_exp.csv")

    return avg_df.T


In [115]:
gene_ids = sub_object.var.index.values
    #cluster line even necessary if not returned?
    #clusters = anndataobject.obs[celltype].cat.categories
obs = sub_object[:,gene_ids].X.toarray()
obs = np.expm1(obs)
sub_object = pd.DataFrame(obs,columns=gene_ids,index= sub_object.obs["new_annotation"])
sub_object = sub_object.groupby(level=0, observed=False).mean()
#sub_object.T.to_csv(outpath + name + "average_gene_expression_by_cluster_exp.csv")

sub_object.T


AttributeError: 'function' object has no attribute 'index'

In [48]:
#not per cluster but cluster and pval etc need to be added to csv (check against specific marker csv from lr2tf test run in R)

def get_significant_tfs_single(sub_object, celltype, condition, out_path, pval, logfc):
    
    #does not work if condition is None
    single_result_path = out_path + condition 
    if not os.path.isdir(single_result_path):
        os.mkdir(single_result_path)
    
    #sc.pp.scale(tf_activities)
    #or sc.pp.normalize_total(anndataobject)
    # sc.pp.log1p(anndata_object) 
    

    number_of_clusters = len(sub_object.obs["new_annotation"].cat.categories) 
    
    #//TODO: 
    sc.tl.rank_genes_groups(sub_object.obsm["ulm_estimate"], groupby= "new_annotation", reference="rest", method="wilcoxon", key_added="wilcoxon_markers", corr_method= "bonferroni")
    #sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="wilcoxon_markers", key_added= "wilcoxon_markers_filtered")
     
    sc.tl.rank_genes_groups(sub_object.obsm["ulm_estimate"], groupby= "new_annotation", reference="rest", method="t-test_overestim_var", key_added="t_test_overestim_var_markers")
    #sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="t_test_overestim_var_markers", key_added="t_test_overestim_filtered")

    #FindAllMarkers(seuratobject, only.pos = TRUE, min.pct = 0, logfc.threshold = 0, verbose = FALSE)

    result1 = sub_object.obsm["ulm_estimate"].X.uns['wilcoxon_markers']
    groups = result1['names'].dtype.names
    anndataobject_markers = pd.DataFrame(
    {group + '_' + key[:1]: result1[key][group]
    for group in groups for key in ['names','logfoldchanges','pvals','pvals_adj']})

    result2 = sub_object.obsm["ulm_estimate"].X.uns['t_test_overestim_var_markers']
    groups = result2['names'].dtype.names
    anndataobject_markers = pd.DataFrame(
    {group + '_' + key[:1]: result2[key][group]
    for group in groups for key in ['names','logfoldchanges','pvals','pvals_adj']})

    anndataobject_markers_wilcoxon = sc.get.rank_genes_groups_df(sub_object.obsm["ulm_estimate"].X, group = None, log2fc_min=0, key="wilcoxon_markers")
    anndataobject_markers_wilcoxon.to_csv("specific_markers_wilcoxon_test.csv",index=0)

    anndataobject_markers_t_over = sc.get.rank_genes_groups_df(sub_object.obsm["ulm_estimate"].X, group = None, log2fc_min=0, key="t_test_overestim_var_markers")
    anndataobject_markers_t_over.to_csv("specific_markers_t_test_overestim_test.csv",index=0)

    

    res = list()

    return res

In [117]:
anndataobject = ad.read_h5ad("LR2TF_test_run/anndata_object.h5ad")
subobject = []
for name in anndataobject.obs.protocol.unique():
    subobject = anndataobject[anndataobject.obs.protocol == name]
    print (subobject)

View of AnnData object with n_obs × n_vars = 255 × 11426
    obs: 'nCount_RNA', 'nFeature_RNA', 'Species', 'protocol', 'name', 'percent.mt', 'percent.rb', 'S.Score', 'G2M.Score', 'Phase', 'CC.Difference', 'integrated_snn_res.0.5', 'seurat_clusters', 'integrated_snn_res.0.4', 'annotation', 'new_annotation'
    var: 'name'
    obsm: 'X_pca', 'X_umap'
View of AnnData object with n_obs × n_vars = 243 × 11426
    obs: 'nCount_RNA', 'nFeature_RNA', 'Species', 'protocol', 'name', 'percent.mt', 'percent.rb', 'S.Score', 'G2M.Score', 'Phase', 'CC.Difference', 'integrated_snn_res.0.5', 'seurat_clusters', 'integrated_snn_res.0.4', 'annotation', 'new_annotation'
    var: 'name'
    obsm: 'X_pca', 'X_umap'


In [46]:
#ignore extra tfs from decoupler while writing script 

def tf_activity_analysis (anndataobject, arguments_list = None):
    
    if (isinstance(anndataobject, str)):
        anndataobject = ad.read_h5ad(anndataobject)

    arguments_list = validate_input_arguments(arguments_list)

    if not os.path.isdir(arguments_list["out_path"]):
        os.mkdir(arguments_list["out_path"])
        tf_path = arguments_list["out_path"] + "TF_results/"
        os.mkdir(tf_path)
    else:
        tf_path = arguments_list["out_path"] + "TF_results/"

    #rename the arguments inserted into argument list (eg protocol to condition)

    #checks for tf activity csv, if nothing there runs decoupler
    if isinstance(arguments_list["tf_activities"], str):
        tf_activities = ad.read_csv(arguments_list["tf_activities"])
        anndataobject.obsm['ulm_estimate'] = tf_activities 
    elif arguments_list["tf_activities"] is None:
         dc.run_ulm(mat = anndataobject, net = "reg", source ='source', target ='target', weight ='weight', verbose = True, use_raw = False)
         tf_activities = anndataobject.obsm['ulm_estimate']
         tf_activities.to_csv("decoupler_results.csv")


    #sets the stage for decision if single condition or comparison analysis is done

    if not np.isnan(arguments_list["comparison_list"]):
        if len(arguments_list["comparison_list"]) > 0 & len(anndataobject.obs["comparison_list"]) < 2:
            arguments_list["comparison_list"] <- np.nan
            print("Only one condition was found in the data, although a list of comparisons was provided. The analyses are performed only for the present condition!")

    #code for single condition  analysis

    if np.isnan(arguments_list["comparison_list"]):
        result_list = []
        gene_expression_list = []
        CTR_cluster_list = []
        intranet_cluster_list = []

        #creates loop until after tf activity score

        sub_object = []
        for name_iterable in anndataobject.obs.protocol.unique():
            sub_object = anndataobject[anndataobject.obs.protocol == name_iterable]

            #name <- str_replace_all(name, "[,;.:-]", "_")
            # sub_object.uns["Average_Expression"] gives error
            sub_object_avg = AverageExpression(sub_object, name_iterable= name_iterable, celltype = arguments_list["celltype"], outpath= arguments_list["out_path"])
            #check if its fine to only have the average expression as a dataframe and not as part of an anndata object

            #check how sub object is returned by average expression (subobject.T) or rename return?
            tf_activity_scores = get_significant_tfs_single(sub_object, arguments_list["celltype"], arguments_list["condition"], tf_path, pval = arguments_list["pval"], logfc = arguments_list["logfc"])
            #result_list["name"] = tf_activity_scores
            #gene_expression_list[name + "_average_expression"] = sub_object.layers["cell_type"]
            #if (arguments_list["organism"] == "human"):
            #CTR_cluster_list["name"] = generate_CrossTalkeR_input(tf_activity_scores[
            # intranet_cluster_list[[name]] 

        return(sub_object) #return tf when finished

In [49]:
result = tf_activity_analysis(anndataobject= "LR2TF_test_run/anndata_object.h5ad", arguments_list= {"out_path" : "script_test", "celltype" : "new_annotation", "condition" : "protocol", "organism" : None, "comparison_list" : None, "logfc" : None, "pval" : None, "reg" : "filterd_regulon.csv", "tf_activities" : "decoupler_results.csv"})

NotImplementedError: No view type has been registered for <class 'anndata._core.anndata.AnnData'>

In [ ]:
#current errors:

#Avg expression
#NotImplementedError: No view type has been registered for <class 'anndata._core.anndata.AnnData'>
